📌 Objetivo: aplicar transformações reutilizáveis com qualidade de produção

✅ CÉLULA 1 – Setup e bibliotecas

#### 📚 BIBLIOTECAS PARA TRANSFORMAÇÃO DE DADOS
#### Projeto: Dataflow Sentinel – Transformação Modular

In [ ]:
# =============================================================
# 📚 BIBLIOTECAS PARA TRANSFORMAÇÃO DE DADOS
# Projeto: Dataflow Sentinel – Transformação Modular
# =============================================================

import pandas as pd
import numpy as np
import logging
import os

# Logging
logging.basicConfig(level=logging.INFO,
                    format='[%(levelname)s] %(asctime)s - %(message)s',
                    datefmt='%H:%M:%S')
logger = logging.getLogger()


✅ CÉLULA 2 – Leitura dos dados validados

#### 📁 LEITURA DO DATASET VALIDADO (.parquet)

In [ ]:
# =============================================================
# 📁 LEITURA DO DATASET VALIDADO (.parquet)
# =============================================================

parquet_path = "/content/dataflow_validado.parquet"

if not os.path.exists(parquet_path):
    logger.error(f"Arquivo Parquet não encontrado: {parquet_path}")
else:
    logger.info(f"Arquivo localizado: {parquet_path}")

df = pd.read_parquet(parquet_path)
df.head()


,shipment_id,route_id,date,delay_minutes,cost_usd,temp_c,status
0,SHP10000,RT02,1998-01-08,12,36.23,7.4,delivered
1,SHP10001,RT03,1998-01-09,8,37.28,4.4,failed
2,SHP10002,RT04,1998-01-10,7,31.80,3.1,in_transit
3,SHP10003,RT04,1998-01-11,12,98.73,5.1,in_transit
4,SHP10004,RT02,1998-01-12,10,69.52,0.8,delivered


✅ CÉLULA 3 – Conversão de date e criação de partes de data

#### 🗓️ CONVERSÃO DE DATAS + EXTRAÇÃO TEMPORAL

In [ ]:
# =============================================================
# 🗓️ CONVERSÃO DE DATAS + EXTRAÇÃO TEMPORAL
# =============================================================

df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Criar colunas derivadas de tempo
df['ano'] = df['date'].dt.year
df['mes'] = df['date'].dt.month
df['dia'] = df['date'].dt.day
df['dia_semana'] = df['date'].dt.day_name()
df['fim_de_semana'] = df['date'].dt.weekday >= 5  # True para sábado e domingo

logger.info("✅ Colunas temporais adicionadas.")
df[['date', 'ano', 'mes', 'dia', 'dia_semana', 'fim_de_semana']].head()


,date,ano,mes,dia,dia_semana,fim_de_semana
0,1998-01-08,1998,1,8,Thursday,False
1,1998-01-09,1998,1,9,Friday,False
2,1998-01-10,1998,1,10,Saturday,True
3,1998-01-11,1998,1,11,Sunday,True
4,1998-01-12,1998,1,12,Monday,False


✅ CÉLULA 4 – Feature Engineering adicional

#### 🧠 FEATURE ENGINEERING: FLAGS DE NEGÓCIO

In [ ]:
# =============================================================
# 🧠 FEATURE ENGINEERING: FLAGS DE NEGÓCIO
# =============================================================

# Flag para atraso crítico (ex: > 15 minutos)
df['atraso_critico'] = df['delay_minutes'] > 15

# Classificação de custo
df['custo_categoria'] = pd.cut(df['cost_usd'],
                               bins=[0, 50, 75, np.inf],
                               labels=['baixo', 'médio', 'alto'])

logger.info("🛠️ Flags e categorias de negócio adicionadas.")
df[['delay_minutes', 'atraso_critico', 'cost_usd', 'custo_categoria']].head()


,delay_minutes,atraso_critico,cost_usd,custo_categoria
0,12,False,36.23,baixo
1,8,False,37.28,baixo
2,7,False,31.80,baixo
3,12,False,98.73,alto
4,10,False,69.52,médio


✅ CÉLULA 5 – Função transforma_dados(df)

#### 🔄 FUNÇÃO DE TRANSFORMAÇÃO REUTILIZÁVEL

In [ ]:
# =============================================================
# 🔄 FUNÇÃO DE TRANSFORMAÇÃO REUTILIZÁVEL
# =============================================================

def transforma_dados(df: pd.DataFrame) -> pd.DataFrame:
    """
    Aplica transformações padronizadas no dataset validado.
    Inclui conversão de datas e geração de colunas úteis para ML.
    """
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df['ano'] = df['date'].dt.year
    df['mes'] = df['date'].dt.month
    df['dia'] = df['date'].dt.day
    df['dia_semana'] = df['date'].dt.day_name()
    df['fim_de_semana'] = df['date'].dt.weekday >= 5
    df['atraso_critico'] = df['delay_minutes'] > 15
    df['custo_categoria'] = pd.cut(df['cost_usd'],
                                   bins=[0, 50, 75, np.inf],
                                   labels=['baixo', 'médio', 'alto'])
    logger.info("✅ Transformações aplicadas com sucesso.")
    return df


✅ CÉLULA FINAL – Aplicar e salvar

#### 💾 APLICAR TRANSFORMAÇÃO E SALVAR

In [ ]:
# =============================================================
# 💾 APLICAR TRANSFORMAÇÃO E SALVAR
# =============================================================

df_transformado = transforma_dados(df)

output_path = "/content/dataflow_transformado.parquet"
df_transformado.to_parquet(output_path, index=False)
logger.info(f"📦 Dataset transformado salvo em: {output_path}")

# Preview
df_transformado.head()


,shipment_id,route_id,date,delay_minutes,cost_usd,temp_c,status,ano,mes,dia,dia_semana,fim_de_semana,atraso_critico,custo_categoria
0,SHP10000,RT02,1998-01-08,12,36.23,7.4,delivered,1998,1,8,Thursday,False,False,baixo
1,SHP10001,RT03,1998-01-09,8,37.28,4.4,failed,1998,1,9,Friday,False,False,baixo
2,SHP10002,RT04,1998-01-10,7,31.80,3.1,in_transit,1998,1,10,Saturday,True,False,baixo
3,SHP10003,RT04,1998-01-11,12,98.73,5.1,in_transit,1998,1,11,Sunday,True,False,alto
4,SHP10004,RT02,1998-01-12,10,69.52,0.8,delivered,1998,1,12,Monday,False,False,médio


✅ Markdown final

### 🔄 Etapa Concluída – Transformação de Dados

Este notebook aplica transformações temporais e flags estratégicas no dataset validado:

- Conversão e decomposição de datas
- Flags de negócio para ML (ex: atraso crítico, custo alto)
- Dados prontos para consumo por modelos

📤 Saída:
- `dataflow_transformado.parquet`
- Função `transforma_dados(df)` para uso em batch/pipeline

➡️ Pronto para o notebook 04: modelo leve com scikit-learn
